In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# %cd ..

C:\Users\tzeng1\repos\LIDC_GDRO


In [57]:
import pandas as pd
# import numpy as np
from dataloaders import InfiniteDataLoader
from datasets import NoduleDataset, SubclassedNoduleDataset
from models import TransferModel
from loss import ERMLoss#, GDROLossAlt, GDROLoss
from train import train, test
from torch.optim.lr_scheduler import ReduceLROnPlateau

# import torchvision
from torchvision import transforms
import torch
from image_data_utils import getImages

In [5]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Using cpu


In [70]:
train, cv, test = getImages(split=True, sublabels=True, binary=True, device=DEVICE)

147

In [48]:
y = [torch.tensor(int(i > 1), device=DEVICE) for i in img_data[1]]

In [49]:
img_data = (X,y,c)

In [50]:
import random
shuffle_data = list(zip(*img_data, nodule_id))
random.shuffle(shuffle_data)
s_d = list(zip(*shuffle_data))
shuffle_data, shuffle_id = s_d[:3], s_d[3]

In [37]:
len(shuffle_data)

3

In [51]:
tr = SubclassedNoduleDataset(shuffle_data[0][:744],shuffle_data[1][:744], shuffle_data[2][:744])
cv = SubclassedNoduleDataset(shuffle_data[0][744:],shuffle_data[1][744:], shuffle_data[2][744:])
loader = InfiniteDataLoader(tr, batch_size=128)
cv_loader = InfiniteDataLoader(cv, len(cv))

In [53]:
model = TransferModel(device=DEVICE, pretrained=True, freeze=False)
loss_fn = ERMLoss(model,torch.nn.CrossEntropyLoss(),{}, subclassed=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.005)#lr=0.001, weight_decay=0.005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=2, verbose=True)


#train function

In [54]:
def train_epochs(epochs, train_loader, val_loader, model, loss_fn, optimizer,scheduler, verbose=True):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        train(train_loader, model, loss_fn, optimizer, verbose=verbose)
        accuracies = test(val_loader, model, verbose=verbose)
        scheduler.step(min(accuracies[1:]))

In [55]:
train_epochs(15,loader,cv_loader,model,loss_fn,optimizer,scheduler)

Epoch 1/15
Average training loss: 0.7341472089290619
Accuracy: 0.657258064516129 
Accuracy over subgroups: [0.94827586 0.96727273 0.12903226 0.31192661] 
Worst Group Accuracy: 0.12903225806451613
Epoch 2/15
Average training loss: 0.2848806917667389
Accuracy: 0.760752688172043 
Accuracy over subgroups: [0.9760479  0.98154982 0.32608696 0.63114754] 
Worst Group Accuracy: 0.32608695652173914
Epoch 3/15
Average training loss: 0.17363651618361473
Accuracy: 0.7567204301075269 
Accuracy over subgroups: [0.88304094 0.94945848 0.2866242  0.74820144] 
Worst Group Accuracy: 0.28662420382165604
Epoch 4/15
Average training loss: 0.1163276806473732
Accuracy: 0.8293010752688172 
Accuracy over subgroups: [0.93956044 0.875      0.58730159 0.94871795] 
Worst Group Accuracy: 0.5873015873015873
Epoch 5/15
Average training loss: 0.06615827903151512
Accuracy: 0.8198924731182796 
Accuracy over subgroups: [0.84153005 0.87956204 0.5952381  0.96638655] 
Worst Group Accuracy: 0.5952380952380952
Epoch 6/15
Averag